## run this cell before starting spark cluster connection (corrects module error)

In [1]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [4]:
df_april = spark.read.json("/user/sprenner/training_official_april.json")

In [5]:
df_mai = spark.read.json("/user/sprenner/training_official_mai.json")

### Create temporary table view

In [6]:
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="hits", outputCol="bucketedFeatures")
bucketedDataApril = bucketizer.transform(df_april)
bucketedDataApril.createOrReplaceTempView("bucketApril")

bucketedDataMai = bucketizer.transform(df_mai)
bucketedDataMai.createOrReplaceTempView("bucketMai")



In [7]:
spark.sql("SELECT bucketedFeatures, count(*) \
            FROM bucketApril \
            GROUP BY  bucketedFeatures \
            ORDER BY 1 ASC").toPandas()

bucketedFeatures  count(1)
0               0.0    425303
1               1.0    351983

In [8]:
spark.sql("SELECT bucketedFeatures, count(*) \
            FROM bucketMai \
            GROUP BY  bucketedFeatures \
            ORDER BY 1 ASC").toPandas()

bucketedFeatures  count(1)
0               0.0    250009
1               1.0    339274

In [9]:
final_data_april = bucketedDataApril.withColumn("class_bucket", when(bucketedDataApril["bucketedFeatures"] == 0.0, "nocache").otherwise("cache"))
final_data_mai = bucketedDataMai.withColumn("class_bucket", when(bucketedDataMai["bucketedFeatures"] == 0.0, "nocache").otherwise("cache"))

In [10]:
final_data_april.createOrReplaceTempView("final_april")
final_data_mai.createOrReplaceTempView("final_mai")

In [11]:
spark.sql("SELECT class_bucket, count(*) \
            FROM final_april \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache    351983
1      nocache    425303

In [12]:
spark.sql("SELECT class_bucket, count(*) \
            FROM final_mai \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache    339274
1      nocache    250009

In [13]:
final_data_april.describe().toPandas()

summary  account          accountHash         clientState  \
0   count   777286               777286              777286   
1    mean     None  1.935684591732567E8                None   
2  stddev     None  1.136131724108707E9                None   
3     min  aagaard          -2127257360  CannotAuthenticate   
4     max   zzheng           2143592175      SourceNotFound   

        clientStateHash                                            dataset  \
0                777286                                             777286   
1   -9.74016245191591E8                                               None   
2  2.6212986830087784E8                                               None   
3           -1922357674  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r...   
4            1269532611  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...   

            datasetHash datasetScope      datasetScopeHash dataset_version  \
0                777286       777286                777286          777286   
1  1.4554549800768314E7         None  -7.552649492442358E8            None   
2  1.1941649925847783E9         None   8.314865501449598E8            None   
3           -2147401420    data10_hi           -2057006107     c1130_m1847   
4            2147445885   mc16_valid            2109319881      x557_m1947   

    dataset_versionHash datatype         datatypeHash eventType  \
0                777286   777286               777286    777286   
1   2.973310495739393E8     None  4.947060556405943E8      None   
2  1.2288188131163642E9     None  1.170030329263279E9      None   
3           -2146716518      AOD          -2070622869  download   
4            2146405934      log           2124512252  download   

          eventTypeHash eventVersion      eventVersionHash  \
0                777286       777286                777286   
1            5.937808E7         None  -1.992468944589716E9   
2  7.009964179056578E-9         None   3.073273964746104E8   
3              59378080       1.13.2           -2044360925   
4              59378080       1.15.5            1261702107   

                             filename          filenameHash  \
0                              777286                777286   
1                                None   -248557.46481861244   
2                                None  1.2402658149710774E9   
3    AOD.05371635._000001.pool.root.1           -2147478094   
4  log.13787275._000072.job.log.tgz.1            2147482617   

       filesize_double                hits                  hostname  \
0               777286              777286                    777286   
1   6.11402028756827E8  2.8777927300890536                      None   
2  1.240877180994578E9  3.9916293684162376                      None   
3                304.0                   1    ReCaS-ui-01.cs.infn.it   
4      1.1991446515E10                  48  xenia.nevis.columbia.edu   

            hostnameHash prod_step          prod_stepHash     project  \
0                 777286    777286                 777286      777286   
1  -1.1804844919942595E8      None  -3.3290311308177426E7        None   
2   1.2286452227051136E9      None   1.1050073037449539E9        None   
3            -2133418459  PhysCont            -1329401808   data10_hi   
4             2129595856     simul             2037678380  mc16_valid   

            projectHash protocol          protocolHash             remoteSite  \
0                777286   777286                777286                 777286   
1  -7.551860864380562E8     None  3.5590511037907284E8                   None   
2   8.315047181989845E8     None  1.0536208601928804E9                   None   
3           -2057006107     davs           -1607963002        AGLT2_CALIBDISK   
4            2109319881      srm            2128034789  ZA-WITS-CORE_PRODDISK   

         remoteSiteHash         run_number        run_numberHash       scope  \
0                777286             777286                777286      777286   
1  -6.18880

In [14]:
final_data_mai.describe().toPandas()

summary   account           accountHash         clientState  \
0   count    589283                589283              589283   
1    mean      None  -7.873822054121874E7                None   
2  stddev      None  1.1303640225554883E9                None   
3     min     abell           -2123889579  CannotAuthenticate   
4     max  zmarshal            2140382084      SourceNotFound   

         clientStateHash                                            dataset  \
0                 589283                                             589283   
1  -1.0635692705599144E9                                               None   
2   2.7545692038495255E8                                               None   
3            -1913538301  data12_8TeV.00200805.physics_MinBias.recon.NTU...   
4              634051254  mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_m...   

             datasetHash datasetScope      datasetScopeHash dataset_version  \
0                 589283       589283                589283          589283   
1  -1.4774623149099838E7         None  -9.356107326362851E8            None   
2   1.2687544178588233E9         None   6.784957656574695E8            None   
3            -2147401420  data12_8TeV           -2057006107     c1128_m1842   
4             2147445885   mc16_valid            2109319881      x563_m1972   

    dataset_versionHash datatype          datatypeHash eventType  \
0                589283   589283                589283    589283   
1  -3.730470822323909E7     None  2.1895068833955333E8      None   
2   1.101465235450291E9     None  1.0743326832603395E9      None   
3           -2144917680      AOD           -2070622869  download   
4            2146973161      log            2124512252  download   

  eventTypeHash eventVersion       eventVersionHash  \
0        589283       589283                 589283   
1    5.937808E7         None  -2.0115120046498694E9   
2           0.0         None    3.165646873326388E8   
3      59378080      1.14.10            -2044360925   
4      59378080       1.16.1             1261702107   

                             filename          filenameHash  \
0                              589283                589283   
1                                None     684033.7623824207   
2                                None  1.2390276453463612E9   
3    AOD.05341490._000446.pool.root.1           -2147471303   
4  log.14217468._003414.job.log.tgz.1            2147478625   

        filesize_double               hits                  hostname  \
0                589283             589283                    589283   
1  1.4339739956710258E9  4.395322790577702                      None   
2  2.1931399445812006E9   6.04169443570645                      None   
3                9576.0                  1    ReCaS-ui-01.cs.infn.it   
4       1.1976018886E10                 85  xenia.nevis.columbia.edu   

           hostnameHash prod_step          prod_stepHash      project  \
0                589283    589283                 589283       589283   
1  -4.071422388835245E7      None  -2.5500199104117885E8         None   
2   9.756384901185802E8      None    9.244506692727853E8         None   
3           -2138306752  PhysCont            -1058503065  data12_8TeV   
4            2129595856     simul             2037678380   mc16_valid   

            projectHash protocol          protocolHash  \
0                589283   589283                589283   
1  -9.356107326362851E8     None  4.4731323734995073E8   
2   6.784957656574695E8     None   1.103659075718831E9   
3           -2057006107     davs           -1607963002   
4            2109319881      srm            2128034789   

                  remoteSite         remoteSiteHash         run_number  \
0                     589283                 589283             589283   
1                       None  -4.1575698659324807E8  350085.3422377267   
2                       None     1.19455249106281E9  47777.01099327567   
3            AGLT2_CALIBD

## Bucketing End

In [15]:

from pyspark.ml.classification import MultilayerPerceptronClassifier

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

assembler = VectorAssembler(inputCols=["accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash",
                                       "eventVersionHash","filenameHash","filesize_double","hostnameHash","protocolHash",
                                       "remoteSiteHash","scopeHash","traceIdHash","traceIpHash","traceTimeentryUnix",
                                       "transferStart","uuidHash",
                                       "projectHash", "run_numberHash", "stream_nameHash", "prod_stepHash", "datatypeHash",
                                       "dataset_versionHash"], outputCol="features")


output_april = assembler.transform(final_data_april)
#output_april = output_april_all.select("bucketedFeatures", "features")
#output_april = output_april.drop("accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash",
#                                       "eventVersionHash","filenameHash","hostnameHash","protocolHash",
#                                       "remoteSiteHash","scopeHash","traceIdHash","traceIpHash",
#                                       "uuidHash")
output_mai = assembler.transform(final_data_mai)
#output_mai = output_mai_all.select("bucketedFeatures", "features")
#output_mai = output_mai.drop("accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash",
#                                       "eventVersionHash","filenameHash","hostnameHash","protocolHash",
#                                       "remoteSiteHash","scopeHash","traceIdHash","traceIpHash",
#                                       "uuidHash")
        
# ACHTUNG: Random Split gibt unterschiedliche Ergebnisse mit selbem Seed wenn die Spalten nicht die selben sind!!
# Random also auch von Spalten abhängig und spricht nicht nur die Zeilennummer an 
train_data, test_data = output_april.randomSplit([.8,.2],seed=123)

# specify layers for the neural network:
# input layer of size 23 (features), two intermediate of size 8 and 4
# and output of size 2 (classes)
layers = [23, 8, 4, 2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(featuresCol="features", labelCol="bucketedFeatures",maxIter=100, layers=layers, blockSize=128, seed=1234)

model = trainer.fit(train_data)




predictions_april = model.transform(test_data)
predictions_mai = model.transform(output_mai)

# possible metrics: f1|weightedPrecision|weightedRecall|accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="bucketedFeatures", predictionCol="prediction", metricName="accuracy")
accuracy_april = evaluator.evaluate(predictions_april)
print("Test Error April = %g" % (1.0 - accuracy_april))

accuracy_mai = evaluator.evaluate(predictions_mai)
print("Test Error Mai = %g" % (1.0 - accuracy_mai))

#print(str(model.featureImportances))

Test Error April = 0.374885
Test Error Mai = 0.455767


In [20]:
#model.featureImportances

SparseVector(23, {0: 0.0899, 1: 0.38, 2: 0.0041, 3: 0.0364, 5: 0.0016, 6: 0.0, 7: 0.1384, 8: 0.0136, 9: 0.0242, 10: 0.0077, 11: 0.0328, 12: 0.0, 13: 0.0266, 14: 0.0253, 15: 0.0248, 16: 0.0079, 17: 0.0393, 18: 0.0045, 19: 0.0066, 20: 0.0282, 21: 0.075, 22: 0.0331})

In [21]:
predictions_april.limit(10).toPandas()

account  accountHash         clientState  clientStateHash  \
0   abuzatu    154527496                DONE       -878527175   
1   abuzatu    154527496                DONE       -878527175   
2   abuzatu    154527496                DONE       -878527175   
3  acanesse  -1913829851  ServiceUnavailable      -1394468120   
4  acanesse  -1913829851  ServiceUnavailable      -1394468120   
5  acanesse  -1913829851  ServiceUnavailable      -1394468120   
6  acukierm   1777994965                DONE       -878527175   
7  acukierm   1777994965                DONE       -878527175   
8  acukierm   1777994965                DONE       -878527175   
9  acukierm   1777994965                DONE       -878527175   

                                             dataset  datasetHash  \
0  mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttba...   -750525813   
1  mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttba...   -750525813   
2  mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttba...   -750525813   
3  mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTE...   1105772576   
4  mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTE...   1105772576   
5  mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTE...   1105772576   
6  mc15_13TeV.343981.PowhegPythia8EvtGen_NNLOPS_n...    746881300   
7  mc15_13TeV.343981.PowhegPythia8EvtGen_NNLOPS_n...    746881300   
8  mc15_13TeV.343981.PowhegPythia8EvtGen_NNLOPS_n...    746881300   
9  mc15_13TeV.343981.PowhegPythia8EvtGen_NNLOPS_n...    746881300   

  datasetScope  datasetScopeHash                dataset_version  \
0   mc16_13TeV       -1447489496  e5458_s3126_r9781_r9778_p3384   
1   mc16_13TeV       -1447489496  e5458_s3126_r9781_r9778_p3384   
2   mc16_13TeV       -1447489496  e5458_s3126_r9781_r9778_p3384   
3   mc16_13TeV       -1447489496       e3601_e5984_s3126_r10244   
4   mc16_13TeV       -1447489496       e3601_e5984_s3126_r10244   
5   mc16_13TeV       -1447489496       e3601_e5984_s3126_r10244   
6   mc15_13TeV         337785635                          e5607   
7   mc15_13TeV         337785635                          e5607   
8   mc15_13TeV         337785635                          e5607   
9   mc15_13TeV         337785635                          e5607   

   dataset_versionHash     datatype  datatypeHash eventType  eventTypeHash  \
0          -1787879578  NTUP_PILEUP    1967069222  download       59378080   
1          -1787879578  NTUP_PILEUP    1967069222  download       59378080   
2          -1787879578  NTUP_PILEUP    1967069222  download       59378080   
3            157220216          RDO    -134998320  download       59378080   
4            157220216          RDO    -134998320  download       59378080   
5            157220216          RDO    -134998320  download       59378080   
6           2055325324          TXT    -339535536  download       59378080   
7           2055325324          TXT    -339535536  download       59378080   
8           2055325324          TXT    -339535536  download       59378080   
9           2055325324          TXT    -339535536  download       59378080   

  eventVersion  eventVersionHash                                  filename  \
0      1.14.11       -2044360925  NTUP_PILEUP.12628104._000051.pool.root.1   
1      1.14.11       -2044360925  NTUP_PILEUP.12628104._000055.pool.root.1   
2      1.14.11       -2044360925  NTUP_PILEUP.12628104._000060.pool.root.1   
3      1.14.11       -2044360925          RDO.13121605._000024.pool.root.1   
4      1.14.11       -2044360925          RDO.13121605._000026.pool.root.1   
5      1.14.11       -2044360925          RDO.13121605._000046.pool.root.1   
6      1.14.11       -2044360925             TXT.10248297._000071.tar.gz.1   
7      1.14.11       -2044360925             TXT.10248297._000096.tar.gz.1   
8      1.14.11       -2044360925             TXT.10248297._000119.tar.gz.1   
9      1.14.11       -2044360925             TXT.10248297._000144.tar.gz.1   

   filenameHash  filesize_double  hits           hostname  hostnameHash 

In [22]:
predictions_april = predictions_april.withColumn("filesize", predictions_april["filesize_double"].cast(IntegerType()))

In [23]:
predictions_mai = predictions_mai.withColumn("filesize", predictions_mai["filesize_double"].cast(IntegerType()))

In [30]:
predictions_mai_compare = predictions_mai.withColumn("comparison", when(predictions_mai["bucketedFeatures"] == predictions_mai["prediction"], "same").otherwise("different"))
predictions_mai_compare.createOrReplaceTempView("compare")
spark.sql("SELECT comparison, count(*) \
            FROM compare \
            GROUP BY  comparison \
            ORDER BY 1 ASC").toPandas()

comparison  count(1)
0  different     94705
1       same    494578

In [24]:
df_april_sim_eval = predictions_april.select('uuid', 'account', 'dataset', 'filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_april.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction').sort(col("day").asc())

In [25]:
df_mai_sim = predictions_mai.select('uuid', 'account', 'dataset', 'filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_mai.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction').sort(col("day").asc())

In [26]:
df_april_sim_eval.limit(10).toPandas()

uuid   account  \
0  4fbe4bc769124f3d8a0a1425c93fd800    pgadow   
1  960916410a744ba9a886170039597bb3    pgadow   
2  815d9ae39ed54637bc3d8ed2e0ad87e1    pgadow   
3  ab94fcdee5254d068d9115476bcd9780  changqia   
4  c72dd07305fe4090bc52ea59a16b4f12    pgadow   
5  ab94fcdee5254d068d9115476bcd9780  changqia   
6  c9340a493f764a9faa638c9679651a9e    pgadow   
7  ab94fcdee5254d068d9115476bcd9780  changqia   
8  7941cb9221ea49699d43f35cea3c096b    pgadow   
9  f64e3bfe574a40bd81dc94be4fa03b89    pgadow   

                                             dataset  \
0  mc16_13TeV.341044.PowhegPythia8EvtGen_CT10_AZN...   
1  mc16_13TeV.344180.MGPy8EG_PDF4LHC15_A14NNPDF23...   
2  mc16_13TeV.341344.PowhegPythia8EvtGen_CT10_AZN...   
3  data16_13TeV.00300655.physics_Main.deriv.DAOD_...   
4  mc16_13TeV.341504.PowhegPythia8EvtGen_CT10_AZN...   
5  data16_13TeV.00300655.physics_Main.deriv.DAOD_...   
6  mc16_13TeV.342282.PowhegPythia8EvtGen_CT10_AZN...   
7  data16_13TeV.00300655.physics_Main.deriv.DAOD_...   
8  mc16_13TeV.343380.PowhegPythia8EvtGen_CT10_AZN...   
9  mc16_13TeV.343984.MadGraphPythia8EvtGen_A14NNP...   

                                    filename eventType         clientState  \
0   NTUP_PILEUP.13574807._000001.pool.root.1  download                DONE   
1   NTUP_PILEUP.13203411._000085.pool.root.1  download                DONE   
2   NTUP_PILEUP.13604155._000001.pool.root.1  download                DONE   
3  DAOD_HIGG5D1.12610361._000101.pool.root.1  download                DONE   
4   NTUP_PILEUP.12981827._000001.pool.root.1  download                DONE   
5  DAOD_HIGG5D1.12610361._000088.pool.root.1  download  ServiceUnavailable   
6   NTUP_PILEUP.12973007._000026.pool.root.1  download                DONE   
7  DAOD_HIGG5D1.12610361._000101.pool.root.1  download  ServiceUnavailable   
8   NTUP_PILEUP.13576326._000001.pool.root.1  download                DONE   
9   NTUP_PILEUP.12973693._000001.pool.root.1  download                DONE   

          day  traceTimeentryUnix    filesize  prediction  
0  2018-04-01        1.522618e+09       10305         0.0  
1  2018-04-01        1.522619e+09        9800         1.0  
2  2018-04-01        1.522618e+09       10454         1.0  
3  2018-04-01        1.522613e+09  2147483647         0.0  
4  2018-04-01        1.522618e+09       14620         0.0  
5  2018-04-01        1.522603e+09  2147483647         1.0  
6  2018-04-01        1.522618e+09       10395         1.0  
7  2018-04-01        1.522604e+09  2147483647         1.0  
8  2018-04-01        1.522618e+09       10659         1.0  
9  2018-04-01        1.522618e+09       22345         1.0

In [27]:
df_mai_sim.limit(10).toPandas()

uuid account  \
0  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
1  fcd9941419c54995bf2c9e5ece70cd41   cylin   
2  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
3  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
4  fcd9941419c54995bf2c9e5ece70cd41   cylin   
5  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
6  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
7  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
8  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
9  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   

                                             dataset  \
0  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
1  mc16_13TeV.361022.Pythia8EvtGen_A14NNPDF23LO_j...   
2  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
3  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
4  mc16_13TeV.361022.Pythia8EvtGen_A14NNPDF23LO_j...   
5  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
6  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
7  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
8  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
9  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   

                           filename eventType         clientState         day  \
0  AOD.12911492._000209.pool.root.1  download  ServiceUnavailable  2018-05-01   
1  AOD.12911500._000083.pool.root.1  download  ServiceUnavailable  2018-05-01   
2  AOD.12911492._000209.pool.root.1  download  ServiceUnavailable  2018-05-01   
3  AOD.12911492._000209.pool.root.1  download  ServiceUnavailable  2018-05-01   
4  AOD.12911500._000083.pool.root.1  download  ServiceUnavailable  2018-05-01   
5  AOD.12911492._000095.pool.root.1  download  ServiceUnavailable  2018-05-01   
6  AOD.12911492._000095.pool.root.1  download  ServiceUnavailable  2018-05-01   
7  AOD.12911492._000209.pool.root.1  download  ServiceUnavailable  2018-05-01   
8  AOD.12911492._000095.pool.root.1  download  ServiceUnavailable  2018-05-01   
9  AOD.12911492._000095.pool.root.1  download  ServiceUnavailable  2018-05-01   

   traceTimeentryUnix    filesize  prediction  
0        1.525170e+09  2147483647         1.0  
1        1.525169e+09  2147483647         1.0  
2        1.525170e+09  2147483647         1.0  
3        1.525170e+09  2147483647         1.0  
4        1.525169e+09  2147483647         1.0  
5        1.525170e+09  2147483647         1.0  
6        1.525170e+09  2147483647         1.0  
7        1.525170e+09  2147483647         1.0  
8        1.525170e+09  2147483647         1.0  
9        1.525170e+09  2147483647         1.0

In [28]:
#df_april_sim_eval.write.csv('/user/sprenner/forest_april_official__eval_predictions_numTrees300_seed1234_cache01.csv', sep='\t')

In [29]:
df_mai_sim.write.csv('/user/sprenner/forest_mai_official_predictions_numTrees300_seed1234_cache01.csv', sep='\t')